# Importação de bibliotecas

In [ ]:
#import numpy as np

# # Visualização dos dados
# import seaborn as sns
# import matplotlib.pyplot as plt

# %matplotlib inline
# sns.set_theme()

# # Pré-processamento dos dados
# from sklearn.model_selection import train_test_split, TimeSeriesSplit
# from sklearn.preprocessing import StandardScaler

# # Criação de modelos
# from sklearn.dummy import DummyRegressor

# from keras.models import Sequential
# from keras.layers import Dense, SimpleRNN
# from keras.optimizers import SGD, RMSprop
# from keras.callbacks import EarlyStopping

# # Métricas de avaliação
# from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error, mean_absolute_error, mean_squared_log_error


# Escalonamento dos atributos

In [ ]:
scaler = StandardScaler()

X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.transform(X_test)

y_train_norm = scaler.fit_transform(y_train.to_numpy().reshape(-1, 1))
y_test_norm = scaler.transform(y_test.to_numpy().reshape(-1, 1))

In [ ]:
with sns.axes_style("whitegrid"):
    fig, ax = plt.subplots(figsize=(15,7))
    sns.boxplot(data=X_train_norm, palette='Set2')
    ax.set_title('Distribuição do dados com StandardScaler', fontdict={'fontsize':18, 'fontweight': 'bold'})
    plt.xticks([0,1,2,3],['angulo_zenital', 'irradiancia_ceu_claro', 'umidade_relativa',
       'temperatura'])
    plt.show()

# StandardScaler
# <center>$z = \frac{(x - \mu)}{\sigma}$</center>

Onde $x$ são as amostras do conjunto de dados, $u$ a média do astributo e $s$, o desvio padrão.

In [ ]:
# Propriedades estatísticas após normalização dos dados

pd.DataFrame(X_train_norm).describe()

### todo
+ Verificar o makepipeline
+ ver artigo sobre a normalization

# Implementação da rede recorrente de ELMAN

In [ ]:
treinamento = ufopa_[:int(len(ufopa_)*0.8)]
teste = ufopa_[int(len(ufopa_)*0.8):]

In [ ]:
len(treinamento), len(teste)

In [ ]:
treinamento = scaler.fit_transform(treinamento)
teste = scaler.transform(teste)

### Aplicando a janela deslizante

In [ ]:
def sliding_window(data, lag):
    '''
        Função que aplica a técnica de transformação dos dados em função de uma janela de tempo (lag)
    '''
    size_data = len(data)
    X, y = [], []
    for i in range(lag, size_data):
        X.append(data[i-lag:i])
        y.append(data[i])
    return np.asarray(X), np.asarray(y)

In [ ]:
LAG = 13
Xtrain, ytrain = sliding_window(treinamento[:,5], LAG)
Xtest, ytest = sliding_window(teste[:,5], LAG)

In [ ]:
Xtrain = np.reshape(Xtrain, (Xtrain.shape[0], Xtrain.shape[1], 1))
Xtest = np.reshape(Xtest, (Xtest.shape[0], Xtest.shape[1], 1))

Xtrain.shape, Xtest.shape

### todo
+ Adicionar seed

###  Criação do modelo

In [ ]:
model = Sequential(name='Elman_RNN')

model.add(SimpleRNN(units=10,
                    input_shape=(Xtrain.shape[1], Xtrain.shape[2]),
                    activation='relu',
                    name='Recorrent_layer'))

model.add(Dense(units=1,
                activation='linear',
                name='output_layer'))

model.compile(optimizer=SGD(learning_rate=0.125, momentum=0.93),
             loss='mse',
             metrics=['mape'])

callback = EarlyStopping(monitor='loss', patience=10, min_delta=1e-7)

hist = model.fit(Xtrain, ytrain, batch_size=32, epochs=100, callbacks=[callback])

In [ ]:
loss = hist.history['loss']
ax = sns.lineplot(data=loss, linewidth=1.5)
ax.set_title('Função de custo')
ax.set_xlabel('Épocas')
ax.set_ylabel('Erro quadrático médio')
plt.show()

In [ ]:
model.evaluate(Xtest, ytest)

### todo
Adicionar atributos:
+ Sazonalidade (mês, estação do ano)
+ Lag
+ Diff Lag
+ Media móvel

## todo reunião (17/05/2021)

+ verificar a cobertura de nuvens para as duas localidades
+ Fazer um ppt com os resultados
+ Confirmar reunião às 13:30


## Todo

+ Dividir os dados em treino, validação e teste e subir para o github.  
 (Não esquecer de alterar os links dos repositórios nos notebooks)

+ Separar por pastas: local_BR, local_EUA


